# Geographically Weighted PCA

Standard PCA assumes the same correlation structure everywhere in the dataset.
**Geographically Weighted PCA (GWPCA)** relaxes this: for each observation,
a local covariance matrix is estimated using a spatial kernel that down-weights
distant neighbours.  Each point therefore has its own set of principal components
reflecting the correlation structure of its immediate geographic neighbourhood.

This notebook demonstrates GWPCA on a synthetic spatial dataset where four
geographic zones have genuinely different underlying factor structures.  The
local PC scores are then passed to **GeoLatent** to reveal how the latent
embedding space is organised by geography.

In [1]:
!pip install -q "geolatent[umap]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 2.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.preprocessing import StandardScaler
from geolatent import inspect_latent_space, DARK_SCIENTIFIC

pio.renderers.default = "colab"
rng = np.random.default_rng(42)

---
## Spatial Dataset

A 25×25 grid of neighbourhood observations, each described by 8 socioeconomic
features.  Four latent factors are injected, each dominating a different
geographic quadrant:

| Zone | Coordinates | Dominant factor |
|---|---|---|
| NW | x < 0, y > 0 | Factor A — socioeconomic |
| NE | x > 0, y > 0 | Factor B — demographic |
| SW | x < 0, y < 0 | Factor C — infrastructure |
| SE | x > 0, y < 0 | Factor D — environmental |

Global PCA will find an average structure across all zones.  GWPCA should
recover zone-specific structure that global PCA misses.

In [3]:
G = 25
grid = np.linspace(-6, 6, G)
xx, yy = np.meshgrid(grid, grid)
coords = np.column_stack([xx.ravel(), yy.ravel()])  # (625, 2)

def spatial_weight(coords, cx, cy, scale=3.0):
    d2 = (coords[:, 0] - cx)**2 + (coords[:, 1] - cy)**2
    return np.exp(-d2 / (2 * scale**2))

# Four latent factors, each localised to a quadrant
wA = spatial_weight(coords, -3,  3)   # NW
wB = spatial_weight(coords,  3,  3)   # NE
wC = spatial_weight(coords, -3, -3)   # SW
wD = spatial_weight(coords,  3, -3)   # SE

fA = rng.normal(size=len(coords))
fB = rng.normal(size=len(coords))
fC = rng.normal(size=len(coords))
fD = rng.normal(size=len(coords))

# Factor-to-feature loading matrices (distinct per zone)
L = {
    "A": np.array([ 1.0,  0.8,  0.6, -0.4,  0.2,  0.0,  0.0,  0.1]),
    "B": np.array([ 0.1,  0.0,  0.2,  0.9,  0.7, -0.5,  0.3,  0.0]),
    "C": np.array([ 0.0,  0.2, -0.3,  0.1,  0.0,  1.0,  0.8,  0.6]),
    "D": np.array([-0.3,  0.5,  0.1,  0.0,  0.4,  0.2, -0.6,  0.9]),
}

X = (
    np.outer(fA * wA, L["A"]) +
    np.outer(fB * wB, L["B"]) +
    np.outer(fC * wC, L["C"]) +
    np.outer(fD * wD, L["D"]) +
    rng.normal(scale=0.25, size=(len(coords), 8))
)

feature_names = ["income", "education", "employment", "age_med",
                 "density", "transit", "green_space", "air_quality"]

# Geographic zone labels (quadrant)
zone_id = (
    (coords[:, 0] >= 0).astype(int) +
    (coords[:, 1] >= 0).astype(int) * 2
)
zone_names = {0: "SW — Infrastructure", 1: "SE — Environmental",
              2: "NW — Socioeconomic",  3: "NE — Demographic"}

X_scaled = StandardScaler().fit_transform(X)
print(f"Grid: {G}×{G} = {len(coords)} neighbourhoods, {X.shape[1]} features")

Grid: 25×25 = 625 neighbourhoods, 8 features


---
## Geographic Distribution

Sanity check: the first feature plotted as a 2-D spatial map.  The spatial
autocorrelation from the injected factors should be visible.

In [4]:
zone_colors = ["#58a6ff", "#3fb950", "#f78166", "#d2a8ff"]

fig = go.Figure(go.Scatter(
    x=coords[:, 0], y=coords[:, 1],
    mode="markers",
    marker=dict(
        color=X_scaled[:, 0],
        colorscale="Plasma",
        size=8,
        showscale=True,
        colorbar=dict(title="income (std)"),
    ),
))
fig.update_layout(
    title="Spatial distribution — income feature",
    xaxis_title="x", yaxis_title="y",
    paper_bgcolor="#0d1117", plot_bgcolor="#161b22",
    font=dict(color="#e6edf3"),
    width=600, height=550,
)
fig.show()

---
## GWPCA Implementation

For each focal point `i`, a Gaussian kernel assigns observation weights that
decay with geographic distance.  A weighted covariance matrix is formed from
these local observations and decomposed by eigenanalysis.  The result is a
set of local PC scores — one 3-vector per neighbourhood — that describe how
the data varies in the immediate vicinity of that point.

In [5]:
def gwpca(X, coords, bandwidth, n_components=3):
    """
    Geographically Weighted PCA.

    For each point i, compute a weighted covariance matrix using a
    Gaussian kernel of the given bandwidth, then extract the top
    n_components local principal components.

    Returns
    -------
    scores : ndarray (n, n_components)
        Local PC scores at each focal point.
    var_ratios : ndarray (n, n_components)
        Local explained variance ratios.
    """
    n = len(X)
    scores = np.zeros((n, n_components))
    var_ratios = np.zeros((n, n_components))

    for i in range(n):
        d = np.sqrt(((coords - coords[i]) ** 2).sum(axis=1))
        w = np.exp(-0.5 * (d / bandwidth) ** 2)

        w_sum = w.sum()
        mu = (X * w[:, None]).sum(0) / w_sum
        Xc = X - mu
        C = (Xc * w[:, None]).T @ Xc / w_sum

        eigvals, eigvecs = np.linalg.eigh(C)
        order = eigvals.argsort()[::-1]
        eigvals, eigvecs = eigvals[order], eigvecs[:, order]

        scores[i] = Xc[i] @ eigvecs[:, :n_components]
        total = eigvals.clip(0).sum() + 1e-12
        var_ratios[i] = eigvals[:n_components].clip(0) / total

    return scores, var_ratios


# Bandwidth of 2.5 units captures roughly the nearest ~50 neighbours
print("Running GWPCA (625 focal points) ...")
local_scores, local_var = gwpca(X_scaled, coords, bandwidth=2.5)
print(f"Done.  Local scores: {local_scores.shape}")
print(f"Mean local explained variance (PC1): {local_var[:, 0].mean():.3f}")

Running GWPCA (625 focal points) ...
Done.  Local scores: (625, 3)
Mean local explained variance (PC1): 0.513


---
## Global PCA vs GWPCA — Explained Variance

Global PCA explains a fixed fraction of variance.  GWPCA's per-point variance
ratios show where the data has a stronger vs weaker local factor structure.
Zones with a dominant latent factor will show higher local PC1 explained variance.

In [6]:
from sklearn.decomposition import PCA

global_pca = PCA(n_components=3).fit(X_scaled)
print("Global PCA explained variance:")
for i, v in enumerate(global_pca.explained_variance_ratio_):
    print(f"  PC{i+1}: {v:.3f}")

print("\nLocal GWPCA mean explained variance per zone:")
for z, name in zone_names.items():
    mask = zone_id == z
    print(f"  {name:<28} PC1={local_var[mask, 0].mean():.3f}  "
          f"PC2={local_var[mask, 1].mean():.3f}")

# Map of local PC1 explained variance
fig = go.Figure(go.Scatter(
    x=coords[:, 0], y=coords[:, 1],
    mode="markers",
    marker=dict(
        color=local_var[:, 0],
        colorscale="Viridis",
        size=8,
        showscale=True,
        colorbar=dict(title="Local PC1<br>var ratio"),
    ),
))
fig.update_layout(
    title="GWPCA — local PC1 explained variance across space",
    xaxis_title="x", yaxis_title="y",
    paper_bgcolor="#0d1117", plot_bgcolor="#161b22",
    font=dict(color="#e6edf3"),
    width=600, height=550,
)
fig.show()

Global PCA explained variance:
  PC1: 0.283
  PC2: 0.226
  PC3: 0.194

Local GWPCA mean explained variance per zone:
  SW — Infrastructure          PC1=0.547  PC2=0.182
  SE — Environmental           PC1=0.455  PC2=0.204
  NW — Socioeconomic           PC1=0.575  PC2=0.175
  NE — Demographic             PC1=0.482  PC2=0.205


---
## Local PC Score Space — GeoLatent

Each neighbourhood is now a 3-vector of local PC scores.  These scores encode
the neighbourhood's position relative to the dominant variation axes *in its
own geographic vicinity*.  GeoLatent projects this 625-point cloud and colours
it by geographic zone.

If GWPCA is capturing genuine zone-specific structure, the four geographic
zones should form distinct clusters in this local PC embedding space —
something global PCA would blur together.

In [7]:
inspect_latent_space(
    local_scores, zone_id,
    config=DARK_SCIENTIFIC.with_method("pca"),
    show_ellipsoids=True,
    show_convex_hulls=True,
    ellipsoid_confidence=0.80,
    class_names=zone_names,
    title="GWPCA local scores — PCA projection (coloured by zone)",
).show()

Compare with the global PCA scores on the same data — all zones share the
same projection axes and the zone-specific structure is less pronounced.

In [8]:
global_scores = global_pca.transform(X_scaled)

inspect_latent_space(
    global_scores, zone_id,
    config=DARK_SCIENTIFIC.with_method("pca"),
    show_ellipsoids=True,
    show_convex_hulls=True,
    ellipsoid_confidence=0.80,
    class_names=zone_names,
    title="Global PCA scores — same data (coloured by zone)",
).show()

---
## Bandwidth Sensitivity

The bandwidth controls the spatial scale of the local analysis.  A narrow
bandwidth gives a highly local picture (sensitive to noise); a wide bandwidth
approaches the global PCA result.  Three bandwidths are shown to illustrate
the transition.

In [9]:
for bw in (1.0, 2.5, 6.0):
    print(f"Bandwidth {bw} ...")
    sc, _ = gwpca(X_scaled, coords, bandwidth=bw)
    inspect_latent_space(
        sc, zone_id,
        config=DARK_SCIENTIFIC.with_method("pca"),
        show_ellipsoids=True,
        class_names=zone_names,
        title=f"GWPCA local scores — bandwidth = {bw}",
    ).show()

Bandwidth 1.0 ...


Bandwidth 2.5 ...


Bandwidth 6.0 ...


---
## t-SNE and UMAP on Local Scores

t-SNE and UMAP reveal the neighbourhood structure of the local score manifold.
Since geographically adjacent points have correlated local covariance matrices,
this manifold should be spatially smooth — the visualisations will show whether
the zones form tight, well-separated clusters or a continuous gradient.

In [10]:
scores_main, _ = gwpca(X_scaled, coords, bandwidth=2.5, n_components=6)

for method in ("tsne", "umap"):
    inspect_latent_space(
        scores_main, zone_id,
        config=DARK_SCIENTIFIC.with_method(method),
        show_ellipsoids=True,
        class_names=zone_names,
        title=f"GWPCA local scores (6-D) — {method.upper()}",
    ).show()